In [ ]:
!pip install opencv-python

In [1]:
import os
import shutil
from sqlalchemy import create_engine, event, Table
from sqlalchemy.orm import sessionmaker

import shapely.wkb
import numpy as np
import cv2

from quickannotator.db import db, Project, Image, AnnotationClass, Notification, Tile, Setting, Annotation, SearchCache

In [2]:
db_path = "sqlite:////opt/QuickAnnotator/quickannotator/instance/quickannotator.db"
engine = create_engine(db_path)#,echo=True)

# Initialize Spatialite extension
@event.listens_for(engine, "connect")
def connect(dbapi_connection, connection_record):
    dbapi_connection.enable_load_extension(True)
    dbapi_connection.execute('SELECT load_extension("mod_spatialite")')
    dbapi_connection.execute('SELECT InitSpatialMetaData(1);')



In [3]:
models = [Image, AnnotationClass, Tile]
db.metadata.create_all(bind=engine, tables=[item.__table__ for item in models])

InitSpatiaMetaData() error:"table spatial_ref_sys already exists"


In [4]:
Session = sessionmaker(bind=engine)
session = Session()

In [6]:
# Get all annotation classes
annotation_classes = session.query(AnnotationClass).all()
print(annotation_classes)

[<AnnotationClass 1>, <AnnotationClass 2>, <AnnotationClass 3>]


In [7]:

class_id = 2

# Query all tiles for the current class
tiles = session.query(Tile).filter_by(annotation_class_id=class_id).all() ## filter by having a gt=True attribute
print(tiles)

[<Tile 1>, <Tile 2>, <Tile 3>, <Tile 4>, <Tile 5>, <Tile 6>, <Tile 7>, <Tile 8>, <Tile 9>, <Tile 10>, <Tile 11>, <Tile 12>, <Tile 13>, <Tile 14>, <Tile 15>, <Tile 16>, <Tile 17>, <Tile 18>, <Tile 19>, <Tile 20>, <Tile 21>, <Tile 22>, <Tile 23>, <Tile 24>, <Tile 25>, <Tile 26>, <Tile 27>, <Tile 28>, <Tile 29>, <Tile 30>, <Tile 31>, <Tile 32>, <Tile 33>, <Tile 34>, <Tile 35>, <Tile 36>, <Tile 37>, <Tile 38>, <Tile 39>, <Tile 40>, <Tile 41>, <Tile 42>, <Tile 43>, <Tile 44>, <Tile 45>, <Tile 46>, <Tile 47>, <Tile 48>, <Tile 49>, <Tile 50>, <Tile 51>, <Tile 52>, <Tile 53>, <Tile 54>, <Tile 55>, <Tile 56>, <Tile 57>, <Tile 58>, <Tile 59>, <Tile 60>, <Tile 61>, <Tile 62>, <Tile 63>, <Tile 64>, <Tile 65>, <Tile 66>, <Tile 67>, <Tile 68>, <Tile 69>, <Tile 70>, <Tile 71>, <Tile 72>, <Tile 73>, <Tile 74>, <Tile 75>, <Tile 76>, <Tile 77>, <Tile 78>, <Tile 79>, <Tile 80>, <Tile 81>, <Tile 82>, <Tile 83>, <Tile 84>, <Tile 85>, <Tile 86>, <Tile 87>, <Tile 88>, <Tile 89>, <Tile 90>, <Tile 91>, <Tile 9

In [30]:
from sqlalchemy import inspect

for tile in tiles:
    image_id = tile.image_id
    gtpred = 'gt'  # or 'pred' based on your requirement
    table_name = f"{image_id}_{class_id}_{gtpred}_annotation"

    # Check if the table exists
    inspector = inspect(engine)
    if not inspector.has_table(table_name):
        continue

    table = Table(table_name, db.metadata, autoload_with=engine)

    # annotations = session.query(table).filter(
    #     table.c.geom.ST_Within(
    #         func.BuildMBR(
    #             tile.geom.bounds[0],
    #             tile.geom.bounds[1],
    #             tile.geom.bounds[2],
    #             tile.geom.bounds[3],
    #             4326  # SRID, adjust if needed
    #         )
    #     )
    # ).all()

    annotations = session.query(table).filter(
        #table.c.polygon.ST_Within(shapely.wkb.dumps(tile.geom))
        table.c.polygon.ST_Within(tile.geom)
    ).all()
    
    if len(annotation) == 0:
        continue

    print("non zero")
    tpoly=shapely.wkb.loads(tile.geom.data)

    # Get the bounds of the polygon
    minx, miny, maxx, maxy = tpoly.bounds

    # Compute the width and height
    width = int(maxx - minx)
    height = int(maxy - miny)

    print(f"Width: {width}, Height: {height}")


    mask = np.zeros((height, width), dtype=np.uint8)

    for annotation in annotations:
        annotation_polygon = shapely.wkb.loads(annotation.geom)
        
        # Translate the annotation polygon to the tile's coordinate system
        translated_polygon = shapely.affinity.translate(
            annotation_polygon, xoff=-tile.geom.bounds[0], yoff=-tile.geom.bounds[1]
        )
        
        # Draw the translated polygon on the mask
        cv2.fillPoly(mask, [np.array(translated_polygon.exterior.coords, dtype=np.int32)], 1)
    
    
    # # Convert the tile geometry to a shapely polygon
    # polygon = shapely.wkb.loads(tile.geom.data)

    
    # # Draw the polygon on the mask
    # cv2.fillPoly(mask, [np.array(polygon.exterior.coords, dtype=np.int32)], 1)

    # # Save or process the mask as needed
    # # For example, save the mask as an image file
    mask_filename = f"mask_{tile.id}.png"
    cv2.imwrite(mask_filename, mask)

session.close()



non zero
Width: 2048, Height: 2048
non zero
Width: 2048, Height: 2048
non zero
Width: 2048, Height: 2048
non zero
Width: 2048, Height: 2048
non zero
Width: 2048, Height: 2048
non zero
Width: 2048, Height: 2048
non zero
Width: 2048, Height: 2048
non zero
Width: 2048, Height: 2048
non zero
Width: 2048, Height: 2048
non zero
Width: 2048, Height: 2048
non zero
Width: 2048, Height: 2048
non zero
Width: 2048, Height: 2048
non zero
Width: 2048, Height: 2048
non zero
Width: 2048, Height: 2048
non zero
Width: 2048, Height: 2048
non zero
Width: 2048, Height: 2048
non zero
Width: 2048, Height: 2048
non zero
Width: 2048, Height: 2048
non zero
Width: 2048, Height: 2048
non zero
Width: 2048, Height: 2048
non zero
Width: 2048, Height: 2048
non zero
Width: 2048, Height: 2048
non zero
Width: 2048, Height: 2048
non zero
Width: 2048, Height: 2048
non zero
Width: 2048, Height: 2048
non zero
Width: 2048, Height: 2048
non zero
Width: 2048, Height: 2048
non zero
Width: 2048, Height: 2048
non zero
Width: 2048

AttributeError: geom